Experimento con Regresión Linear

In [1]:
# Importar las bibliotecas necesarias
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import joblib
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Cargar los datos
train_df = pd.read_csv('../EDA/train.csv')
test_df = pd.read_csv('../EDA/test.csv')
validation_df = pd.read_csv('../EDA/val.csv')

# Separar las características y la variable objetivo
train_X = train_df.drop(columns=['WL'])
train_Y = train_df['WL']

test_X = test_df.drop(columns=['WL'])
test_Y = test_df['WL']

validation_X = validation_df.drop(columns=['WL'])
validation_Y = validation_df['WL']

## Experimento con Regresión Logística

Vamos a realizar un experimento usando regresión logística donde variaremos los hiperparámetros para encontrar la mejor configuración. Los hiperparámetros que ajustaremos son:

- C: Inverso de la fuerza de regularización
- solver: Algoritmo a usar en el problema de optimización
- max_iter: Número máximo de iteraciones

In [3]:
# Definir los parámetros para la búsqueda en cuadrícula
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'liblinear', 'saga'],
    'max_iter': [100, 200, 500]
}

# Crear el modelo base
base_model = LogisticRegression(random_state=42)

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Ajustar el modelo usando GridSearchCV
grid_search.fit(train_X, train_Y)

# Mostrar los mejores parámetros y el mejor score
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor accuracy en validación cruzada:", grid_search.best_score_)

Mejores parámetros: {'C': 10, 'max_iter': 100, 'solver': 'liblinear'}
Mejor accuracy en validación cruzada: 0.860210607190473


In [4]:
# Obtener el mejor modelo
best_model = grid_search.best_estimator_

# Evaluar el modelo en el conjunto de validación
val_pred = best_model.predict(validation_X)
print("\nEvaluación en conjunto de validación:")
print("Accuracy:", accuracy_score(validation_Y, val_pred))
print("\nMatriz de Confusión:")
print(confusion_matrix(validation_Y, val_pred))
print("\nReporte de Clasificación:")
print(classification_report(validation_Y, val_pred))


Evaluación en conjunto de validación:
Accuracy: 0.8655913978494624

Matriz de Confusión:
[[162  23]
 [ 27 160]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.86      0.88      0.87       185
           1       0.87      0.86      0.86       187

    accuracy                           0.87       372
   macro avg       0.87      0.87      0.87       372
weighted avg       0.87      0.87      0.87       372



In [5]:
# Evaluar el modelo en el conjunto de prueba
test_pred = best_model.predict(test_X)

print("\nEvaluación en conjunto de prueba:")
print("Accuracy:", accuracy_score(test_Y, test_pred))
print("\nMatriz de Confusión:")
print(confusion_matrix(test_Y, test_pred))
print("\nReporte de Clasificación:")
print(classification_report(test_Y, test_pred))


Evaluación en conjunto de prueba:
Accuracy: 0.851931330472103

Matriz de Confusión:
[[204  37]
 [ 32 193]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.86      0.85      0.86       241
           1       0.84      0.86      0.85       225

    accuracy                           0.85       466
   macro avg       0.85      0.85      0.85       466
weighted avg       0.85      0.85      0.85       466



In [6]:
# Ejemplo de predicción con nuevos datos
new_data = {
    'W': [47], 
    'L': [30], 
    'W_PCT': [0.61], 
    'MIN': [240], 
    'FGM': [46], 
    'FGA': [84], 
    'FG_PCT': [0.548], 
    'FG3M': [22], 
    'FG3A': [40],
    'FG3_PCT': [0.55], 
    'FTM': [12], 
    'FTA': [17], 
    'FT_PCT': [0.706], 
    'OREB': [8], 
    'DREB': [35], 
    'REB': [43], 
    'AST': [25], 
    'STL': [6],
    'BLK': [2], 
    'TOV': [15], 
    'PF': [15], 
    'PTS': [126], 
    'year': [2025], 
    'month': [4], 
    'day': [6], 
    'team_analyzed': [11],
    'local_team': [24],
    'visitor_team': [11]
}

# Convertir los nuevos datos a DataFrame
new_data_df = pd.DataFrame(new_data)

# Realizar la predicción
predicted_result = best_model.predict(new_data_df)
predicted_prob = best_model.predict_proba(new_data_df)

print(f"Predicción: {predicted_result[0]}")
print(f"Probabilidades: {predicted_prob[0]}")

Predicción: 1
Probabilidades: [0.04307014 0.95692986]


In [7]:
# Guardar el modelo
joblib.dump(best_model, 'best_logistic_regression_model.pkl')
print("Modelo guardado como 'best_logistic_regression_model.pkl'")

Modelo guardado como 'best_logistic_regression_model.pkl'


## Análisis de Resultados

La regresión logística es un algoritmo más simple que Random Forest, pero puede ser efectivo para este tipo de clasificación binaria. Algunas observaciones importantes:

1. La búsqueda en cuadrícula nos ayudó a encontrar los mejores hiperparámetros para el modelo.
2. Podemos comparar el rendimiento con el modelo Random Forest anterior para ver qué enfoque funciona mejor.
3. Las probabilidades de predicción nos dan una idea de la confianza del modelo en sus predicciones.

Una ventaja de la regresión logística es que es más interpretable que Random Forest, ya que podemos analizar los coeficientes para entender la importancia relativa de cada característica. Además, la precisión que se obtuvo con este modelo fue de 0.851. Este resultado fue muy parecido al de SVC, estando a tan solo 0.005 por debajo. 